In [1]:
raw_documents = [
    "고양이가 창밖을 바라보고 있다. 고양이는 창밖을 자주 본다.",
    "강아지가 신나게 뛰어놀고 있다. 강아지는 활발하다.",
    "고양이와 강아지는 둘 다 귀엽다. 고양이와 강아지는 친구다.",
    "창밖에는 비가 내리고 있다. 비 오는 날은 분위기 있다.",
    "비 오는 날에는 따뜻한 차가 좋다. 따뜻한 차는 기분을 좋게 한다.",
    "강아지는 비 오는 날에도 밖에서 뛰어논다. 강아지는 비를 좋아한다.",
    "고양이는 주로 창가에서 낮잠을 잔다. 낮잠을 자는 고양이는 평화롭다."
]

##### ♦️1. 불용어 사전을 사용한 def 함수를 만들어보자
  - 조건 : 키위로 스페이스 보정 / okt로 정규화 적용 및 포스태깅 / 불용어사전적용 /'Noun' 'Verb' 'Adjective' 태그만 출력

In [2]:
import pandas as pd
from kiwipiepy import Kiwi
from konlpy.tag import Okt

In [3]:
stopwords = pd.read_csv('./Data/ko-stopwords.csv')

In [4]:
stopwords

,stopwords
0,가
1,가까스로
2,가령
3,각
4,각각
...,...
600,진짜
601,두
602,게
603,요


In [5]:
kiwi = Kiwi()
okt = Okt()

In [6]:
stopwords = set(stopwords['stopwords'])

In [7]:
def pos_tagging(text):
    if isinstance(text, list):
        return list(map(pos_tagging, text))
        
    string = kiwi.space(text)
    tokens  = okt.pos(string, stem= True, norm = True)
    result = [
        word for word, tag in tokens
        if tag in ['Noun','Verb','Adjective'] and word not in stopwords
    ]
    return result



In [8]:
result_doc = pos_tagging(raw_documents)

In [9]:
# 함께짠 코드
def pos_tagging(documents): #input data가 리스트
    result = []

    for doc in documents: 
        string = kiwi.space(doc) #띄어쓰기 보정
        pos_words = okt.pos(string, norm=True, stem=True) #포스태깅과 정규화, 어간추출 작업 진행 
        result_words = [word for word, tag in pos_words if tag in {'Noun', 'Verb', 'Adjective'} if word not in stopwords]
        result.append(result_words)
    return result

In [10]:
pos_document = pos_tagging(raw_documents)
pos_document

[['고양이', '창', '밖', '바라보다', '고양이', '창밖', '자주', '보다'],
 ['강아지', '신', '나다', '뛰어놀다', '강아지', '활발하다'],
 ['고양이', '강아지', '귀엽다', '고양이', '강아지', '친구'],
 ['창', '비', '내리다', '비', '오다', '날', '분위기'],
 ['비', '오다', '날', '따뜻하다', '차갑다', '좋다', '따뜻하다', '차다', '기분', '좋다'],
 ['강아지', '비', '오다', '날', '밖', '뛰다', '놓다', '강아지', '비', '좋아하다'],
 ['고양이', '주로', '창가', '낮잠', '자다', '낮잠', '자다', '고양이', '평화롭다']]

In [11]:
#CounterVectorizer의 데이터 형식을 맞춰주기 위해 join 사용
result_document = [' '.join(doc) for doc in pos_document] 
result_document

['고양이 창 밖 바라보다 고양이 창밖 자주 보다',
 '강아지 신 나다 뛰어놀다 강아지 활발하다',
 '고양이 강아지 귀엽다 고양이 강아지 친구',
 '창 비 내리다 비 오다 날 분위기',
 '비 오다 날 따뜻하다 차갑다 좋다 따뜻하다 차다 기분 좋다',
 '강아지 비 오다 날 밖 뛰다 놓다 강아지 비 좋아하다',
 '고양이 주로 창가 낮잠 자다 낮잠 자다 고양이 평화롭다']

## 1. BoW
[BOW의 한계점]
- 단어 순서, 즉 맥락을 고려하지 않음 → 의미가 달라질 수 있음
- 유사한 단어를 구별 하지못함 → "고양이"와 "고양이를"이 다른 단어로 인식됨
- 희소 행렬(Sparse Matrix) 문제 → 데이터가 커지면 벡터가 너무 커짐

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer(analyzer=lambda x: x)
bow = vectorizer.fit_transform(result_doc)

In [14]:
feature_name = vectorizer.get_feature_names_out()
feature_name

array(['강아지', '고양이', '귀엽다', '기분', '나다', '날', '낮잠', '내리다', '놓다', '따뜻하다',
       '뛰다', '뛰어놀다', '바라보다', '밖', '보다', '분위기', '비', '신', '오다', '자다', '자주',
       '좋다', '좋아하다', '주로', '차갑다', '차다', '창', '창가', '창밖', '친구', '평화롭다',
       '활발하다'], dtype=object)

In [15]:
bow_array = bow.toarray()
bow_df = pd.DataFrame(bow_array , columns = feature_name)

In [16]:
bow_df

,강아지,고양이,귀엽다,기분,나다,날,낮잠,내리다,놓다,따뜻하다,...,좋아하다,주로,차갑다,차다,창,창가,창밖,친구,평화롭다,활발하다
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,2,...,0,0,1,1,0,0,0,0,0,0
5,2,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
6,0,2,0,0,0,0,2,0,0,0,...,0,1,0,0,0,1,0,0,1,0


## 2. 원-핫 인코딩
[원-핫 인코딩의 한계점]
- 차원의 저주
- 단어간의 관계 표현 및 의미 반영 불가능 (자동차↔차)

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
feature_name.shape

(32,)

In [19]:
feature_name_onehot = feature_name.reshape(-1,1)

In [20]:
encoder = OneHotEncoder(sparse_output= True)

In [21]:
onehot_result = encoder.fit_transform(feature_name_onehot)

In [22]:
encoder.get_feature_names_out()

array(['x0_강아지', 'x0_고양이', 'x0_귀엽다', 'x0_기분', 'x0_나다', 'x0_날', 'x0_낮잠',
       'x0_내리다', 'x0_놓다', 'x0_따뜻하다', 'x0_뛰다', 'x0_뛰어놀다', 'x0_바라보다',
       'x0_밖', 'x0_보다', 'x0_분위기', 'x0_비', 'x0_신', 'x0_오다', 'x0_자다',
       'x0_자주', 'x0_좋다', 'x0_좋아하다', 'x0_주로', 'x0_차갑다', 'x0_차다', 'x0_창',
       'x0_창가', 'x0_창밖', 'x0_친구', 'x0_평화롭다', 'x0_활발하다'], dtype=object)

In [23]:
#원핫인코딩된 값들을 보기좋게 key값과 value 값으로 매칭
#(기존 vectorizer를 사용했을때 사용자가 보기좋게 데이터 프레임으로 변환한 것과 같이 사전 형태로 변환)
one_hot_dict = dict(zip(encoder.get_feature_names_out(), onehot_result))
one_hot_dict

{'x0_강아지': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_고양이': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_귀엽다': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_기분': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_나다': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_날': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_낮잠': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_내리다': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>,
 'x0_놓다': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 32)>

## 3. N-gram

### 3.1 bi-grams

In [24]:
from nltk import bigrams

In [25]:
result_doc

[['고양이', '창', '밖', '바라보다', '고양이', '창밖', '자주', '보다'],
 ['강아지', '신', '나다', '뛰어놀다', '강아지', '활발하다'],
 ['고양이', '강아지', '귀엽다', '고양이', '강아지', '친구'],
 ['창', '비', '내리다', '비', '오다', '날', '분위기'],
 ['비', '오다', '날', '따뜻하다', '차갑다', '좋다', '따뜻하다', '차다', '기분', '좋다'],
 ['강아지', '비', '오다', '날', '밖', '뛰다', '놓다', '강아지', '비', '좋아하다'],
 ['고양이', '주로', '창가', '낮잠', '자다', '낮잠', '자다', '고양이', '평화롭다']]

In [26]:
list(bigrams(result_doc[0]))

[('고양이', '창'),
 ('창', '밖'),
 ('밖', '바라보다'),
 ('바라보다', '고양이'),
 ('고양이', '창밖'),
 ('창밖', '자주'),
 ('자주', '보다')]

In [27]:
bigram_result = [list(bigrams(doc)) for doc in pos_document]
bigram_result

[[('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('강아지', '신'), ('신', '나다'), ('나다', '뛰어놀다'), ('뛰어놀다', '강아지'), ('강아지', '활발하다')],
 [('고양이', '강아지'),
  ('강아지', '귀엽다'),
  ('귀엽다', '고양이'),
  ('고양이', '강아지'),
  ('강아지', '친구')],
 [('창', '비'),
  ('비', '내리다'),
  ('내리다', '비'),
  ('비', '오다'),
  ('오다', '날'),
  ('날', '분위기')],
 [('비', '오다'),
  ('오다', '날'),
  ('날', '따뜻하다'),
  ('따뜻하다', '차갑다'),
  ('차갑다', '좋다'),
  ('좋다', '따뜻하다'),
  ('따뜻하다', '차다'),
  ('차다', '기분'),
  ('기분', '좋다')],
 [('강아지', '비'),
  ('비', '오다'),
  ('오다', '날'),
  ('날', '밖'),
  ('밖', '뛰다'),
  ('뛰다', '놓다'),
  ('놓다', '강아지'),
  ('강아지', '비'),
  ('비', '좋아하다')],
 [('고양이', '주로'),
  ('주로', '창가'),
  ('창가', '낮잠'),
  ('낮잠', '자다'),
  ('자다', '낮잠'),
  ('낮잠', '자다'),
  ('자다', '고양이'),
  ('고양이', '평화롭다')]]

### 3.2 trigrams

In [28]:
from nltk import trigrams

In [29]:
list(trigrams(result_doc[0]))

[('고양이', '창', '밖'),
 ('창', '밖', '바라보다'),
 ('밖', '바라보다', '고양이'),
 ('바라보다', '고양이', '창밖'),
 ('고양이', '창밖', '자주'),
 ('창밖', '자주', '보다')]

### 3.3 4-grams~

In [30]:
from nltk import ngrams

In [31]:
list(ngrams(result_doc[0] , n = 4))

[('고양이', '창', '밖', '바라보다'),
 ('창', '밖', '바라보다', '고양이'),
 ('밖', '바라보다', '고양이', '창밖'),
 ('바라보다', '고양이', '창밖', '자주'),
 ('고양이', '창밖', '자주', '보다')]

In [32]:
ngram_result  = [list(ngrams(result_doc[0] , n = 2)) for doc in result_doc ]

In [33]:
ngram_result

[[('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')]]